In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
from tensorflow.keras.layers import Conv2D

from preprocessing.data.imagenet_labels import imagenet_labels

In [3]:
from models.ResBlock import resnet50

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)    0           input[0][0]                      
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16, 16, 64)   256         conv2d[0][0]                     
______________________________________________________________________________________________

In [4]:
from modules.TFR_load import TFR_load

In [5]:
# Hyper params

BATCH_SIZE     = 512
NUM_OF_CLASS   = 10
LEARNING_RATE  = 1e-5
COSINE_DECAY   = True
NUM_TRAIN_DATA = 60000
NUM_TEST_DATA  = 10000
EPOCH = 100

TR_STEPS_PER_EPOCH = NUM_TRAIN_DATA//BATCH_SIZE
TE_STEPS_PER_EPOCH = NUM_TEST_DATA//BATCH_SIZE

In [6]:
import tensorflow_datasets as tfds

ds_name = 'cifar10'
builder = tfds.builder(ds_name)

tr_ds, te_ds = builder.as_dataset(split = ['train', 'test'], shuffle_files = True)

tr_ds = tr_ds.batch(BATCH_SIZE)
print(len(tr_ds))
te_ds = te_ds.batch(BATCH_SIZE)
print(len(te_ds))

98
20


In [7]:
out_ch = [16, 32, 64, 128]
filter_size = [(5,5),(5,5),(3,3),(3,3),(3,3)]

In [8]:
model = resnet50

In [9]:
model.weights[0]

<tf.Variable 'conv2d/kernel:0' shape=(7, 7, 3, 64) dtype=float32, numpy=
array([[[[ 1.59694403e-02, -1.55205633e-02, -3.80848907e-02, ...,
          -3.53021659e-02, -2.63964068e-02,  6.83103874e-03],
         [ 1.15213357e-02, -2.42442116e-02, -1.12802610e-02, ...,
          -3.38527188e-03,  1.99232064e-02,  3.71966250e-02],
         [ 1.60284042e-02,  3.70808654e-02, -2.64210626e-03, ...,
           1.32832788e-02,  2.80427821e-02, -3.04306149e-02]],

        [[-2.89085303e-02,  2.29227580e-02, -2.58906763e-02, ...,
          -2.26101018e-02, -4.73652035e-03, -2.23931447e-02],
         [ 6.97094947e-04,  1.02615133e-02, -2.03484185e-02, ...,
          -8.84208828e-03,  1.96211934e-02,  4.46433946e-03],
         [ 2.74018012e-02, -2.15006396e-02,  1.12270080e-02, ...,
          -3.60668600e-02, -4.17248905e-03,  1.25420503e-02]],

        [[ 8.44278559e-03,  1.97559483e-02, -2.23125629e-02, ...,
           2.57745348e-02, -3.78531106e-02,  1.52428485e-02],
         [-2.96512656e-02, 

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=1e-08)
loss = tf.keras.losses.CategoricalCrossentropy()
accuracy = tf.keras.metrics.CategoricalAccuracy()

In [11]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [ ]:
step = 1
for EP in range(EPOCH):
    for tr_example, te_example in zip(tr_ds, te_ds):
        images, labels = tr_example["image"], tr_example["label"]
        with tf.GradientTape() as tape:
            logits = model(images / 255)
            loss_value = loss(tf.one_hot(labels, 10), logits)
            accuracy.update_state(tf.one_hot(labels, 10), logits)
            grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        
        images, labels = te_example["image"], te_example["label"]
        logits = model(images / 255)
        loss_value = loss(tf.one_hot(labels, 10), logits)
        accuracy.update_state(tf.one_hot(labels, 10), logits)
        step += 1
    
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', loss_value, step=step)
        tf.summary.scalar('acc', accuracy.result(), step=step)
        accuracy.reset_state()
    
    
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', loss_value, step=step)
        tf.summary.scalar('acc', accuracy.result(), step=step)
        accuracy.reset_state()
        